# Neighborhoods in Toronto -- part-2

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

## create the dataframe

Use beautifulsoup to do it, and then store the data into 3 lisis, finally use pandas to create the dataframe.

**In this part I use the re library to handle the data**

In [3]:
#I copied the code and pasted it into a local text named canada.html
soup=BeautifulSoup(open('canada.html'),features='html.parser')

In [4]:
#create 3 list
PostalCode=[]
Borough=[]
Neighborhood=[]


In [5]:
for items in soup.find('table').find_all('tr'): # item是一行
    item=items.find_all('td')
    for i in range(len(item)):  #遍历一行的元素
        ##print(item[i].text) #处理每一行第一个元素
        stritem=(str(item[i].text))[1:-1]  #把第一个空格去掉
        PostalCode.append(stritem[0:3])                ##这个是（1）str
        strre=stritem[3:-1] # 去掉前三个元素
        #print(strre)
        if strre=='Not assigned':
            Borough.append('Not assigned')
            Neighborhood.append('Not assigned')
        else:
            try:                                   #使用try防止有不带括号的
                res=re.compile(r'(.*?)[(]', re.S)
                A=(re.findall(res,strre))[0]  #A是Borough(2)str
                Borough.append(A)
                res2=re.compile(r'[()](.*?)[)]', re.S)
                B=(re.findall(res2,strre))[0]  #B是Neibor (3)str
                B=B.replace(' / ',',')   #将B转换为，分割
                Neighborhood.append(B)
            except IndexError: #无neibor
                A=strre
                Borough.append(A)
                B=A
                Neighborhood.append(B)


In [6]:
df=pd.DataFrame({'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"


## Drop the rows with "Not assigned" Borough

In [7]:
df=df[df['Borough']!='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [8]:
df.shape

(103, 3)

**we can select the columns as we saw in the homework sample**

In [9]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
dft = pd.DataFrame(columns=column_names)

post_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in post_list:
    dft = dft.append(df[df["PostalCode"]==postcode], ignore_index=True)
dft

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill,Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford,Maryvale"
7,M9V,Etobicoke,"South Steeles,Silverstone,Humbergate,Jamestown..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,King and Spadina,Railway Lands,Harbou..."


## part 2 begin>>

for the connection reason, I've downloaded the csv document 

In [10]:
dflocation=pd.read_csv('Geospatial_Coordinates.csv')

we can use the loop to make it matched between our 2 dataframes

In [33]:
Latitude=[]
Longitude=[]

for i in range(103):
    for j in range(103):
        if df.iat[i,0] ==dflocation.iat[j,0]:    #loop to find the match position
            Latitude.append(dflocation.iat[j,1])
            Longitude.append(dflocation.iat[j,2])

In [35]:
df['Latitude']=Latitude
df['Longitude']=Longitude
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
6,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


**since the order is not the same as what we saw in our homework sample, I created a test dataframe to see if the data is right** 

In [36]:
column_names = ["PostalCode", "Borough", "Neighborhood","Latitude","Longitude"]
dft = pd.DataFrame(columns=column_names)

post_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in post_list:
    dft = dft.append(df[df["PostalCode"]==postcode], ignore_index=True)
dft

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford,Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles,Silverstone,Humbergate,Jamestown...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,King and Spadina,Railway Lands,Harbou...",43.628947,-79.394420
